Data Collection | Preprocessing

In [56]:

import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt


In [57]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

In [58]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/home/pyarena/code for DL/potato_disease/archive(2)/PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 2152 files belonging to 3 classes.


In [59]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [60]:
len(dataset)

68

In [61]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch[0].numpy())

2022-05-20 11:07:22.651245: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25165824 exceeds 10% of free system memory.


NameError: name 'numpy' is not defined